In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 100)
pd.set_option('display.max_rows', 200)
import warnings
warnings.filterwarnings('ignore')

import gc

In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
abalone = fetch_ucirepo(id=1) 
  
# data (as pandas dataframes)
orig_train = pd.concat([abalone.data.features, abalone.data.targets], axis = 1).rename({
    'Whole_weight' : 'Whole weight',
    'Shucked_weight' : 'Whole weight.1',
    'Viscera_weight' : 'Whole weight.2',
    'Shell_weight' : 'Shell weight'
}, axis = 1)
orig_train.to_csv('original_train.csv')

In [3]:
train = pd.read_csv(r'train.csv', index_col = 'id')
test = pd.read_csv(r'test.csv', index_col = 'id')
submission = pd.read_csv("sample_submission.csv")

In [4]:
train = pd.concat([train, orig_train], ignore_index=True)

In [5]:
def feats(df):
#     df['WW1_Ratio'] = df['Whole weight.1'] / df['Whole weight']
#     df['WW2_Ratio'] = df['Whole weight.2'] / df['Whole weight']
#     df['Shell_Ratio'] = df['Shell weight'] / df['Whole weight']
#     df['ProxyVolume'] = 2 * np.log1p(df["Diameter"]) +  np.log1p(df["Height"])
    df['Height']=np.where(df['Height']==0, 0.005, df['Height'])
    return df

train = feats(train)
test = feats(test)

In [7]:
from openfe import OpenFE, transform

ofe = OpenFE()
features = ofe.fit(data=train.drop('Rings', axis=1), label=train['Rings'], feature_boosting=True, n_jobs=8, task='regression', metric='rmse', n_data_blocks=512)
train_x, test_x = transform(train.drop('Rings', axis=1), test, features, n_jobs=8)

train_x.reset_index(drop=True, inplace=True)
train = pd.concat([train_x, train['Rings']], axis=1)
test = test_x.copy()

train.head()

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001398 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1330
[LightGBM] [Info] Number of data points in the train set: 75833, number of used features: 8
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 9.718698
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[272]	valid_0's rmse: 1.8389
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001357 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1330
[LightGBM] [Info] Number of data points in the train set: 75833, number of used features: 8
[Ligh

100%|██████████| 30/30 [00:19<00:00,  1.57it/s]


80 same features have been deleted.
The number of remaining candidate features is 78
Start stage II selection.


100%|██████████| 26/26 [00:06<00:00,  3.86it/s]


Finish data processing.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016011 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19412
[LightGBM] [Info] Number of data points in the train set: 75833, number of used features: 86
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,autoFE_f_0,autoFE_f_1,autoFE_f_2,autoFE_f_3,autoFE_f_4,autoFE_f_5,autoFE_f_6,autoFE_f_7,autoFE_f_8,autoFE_f_9,autoFE_f_10,autoFE_f_11,autoFE_f_12,autoFE_f_13,autoFE_f_14,autoFE_f_15,autoFE_f_16,autoFE_f_17,autoFE_f_18,autoFE_f_19,autoFE_f_20,autoFE_f_21,autoFE_f_22,autoFE_f_23,autoFE_f_24,autoFE_f_25,autoFE_f_26,autoFE_f_27,autoFE_f_28,autoFE_f_29,autoFE_f_30,autoFE_f_31,autoFE_f_32,autoFE_f_33,autoFE_f_34,autoFE_f_35,autoFE_f_36,autoFE_f_37,autoFE_f_38,autoFE_f_39,autoFE_f_40,autoFE_f_41,autoFE_f_42,autoFE_f_43,autoFE_f_44,autoFE_f_45,autoFE_f_46,autoFE_f_47,autoFE_f_48,autoFE_f_49,autoFE_f_50,autoFE_f_51,autoFE_f_52,autoFE_f_53,autoFE_f_54,autoFE_f_55,autoFE_f_56,autoFE_f_57,autoFE_f_58,autoFE_f_59,autoFE_f_60,autoFE_f_61,autoFE_f_62,autoFE_f_63,autoFE_f_64,autoFE_f_65,autoFE_f_66,autoFE_f_67,autoFE_f_68,autoFE_f_69,autoFE_f_70,autoFE_f_71,autoFE_f_72,autoFE_f_73,autoFE_f_74,autoFE_f_75,autoFE_f_76,autoFE_f_77,Rings
0,0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,2672.0,2.348554,1.368750,0.4430,0.0885,0.489898,-0.0935,0.157544,0.036000,136.0,0.0900,3.666667,0.2400,0.3100,2.242321,0.1900,0.4035,340.0,601.0,0.035397,3.214583,0.5765,0.5685,0.2400,0.243335,5.266212,0.2400,-0.0035,0.141255,0.278630,2.935154,0.1465,2.1400,0.7715,0.700,NaN,1.2015,0.062995,NaN,0.6965,0.610417,0.132000,NaN,0.557356,0.2400,0.2835,NaN,0.725,0.049275,0.106898,0.7900,0.550,0.3285,-1.427116,0.9180,0.3285,-0.3415,0.4405,0.215506,0.7585,0.080575,0.24,0.8785,0.115725,0.430,0.2400,1.0,0.1465,0.1465,0.253438,0.6700,0.2400,0.2400,0.575,1.1000,1.0115,0.2400,1.1375,11
1,0,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,1204.0,2.467249,1.431250,0.6720,0.1380,0.565685,-0.0435,0.837460,0.046400,88.0,0.1750,4.344828,0.3200,0.3100,1.656420,0.1700,0.3535,86.0,40.0,0.035333,3.531250,0.7665,0.7780,0.3200,0.956930,4.086799,0.3200,0.1315,0.224420,0.715119,1.772152,0.2765,1.8075,0.1300,0.775,NaN,1.6200,0.135485,NaN,0.9065,0.864063,0.201600,NaN,0.433628,0.3200,0.2135,NaN,0.720,0.066410,0.963678,0.9500,0.630,0.4580,-1.139434,1.4065,0.4580,-0.6400,0.4695,0.649572,0.9480,0.174195,0.32,1.0880,0.163850,0.490,0.3200,1.0,0.2765,0.2765,0.517540,0.8100,0.3200,0.3200,0.585,1.5880,1.4500,0.3200,0.8760,11
2,1,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,499.0,3.818182,1.100000,0.0155,0.0005,0.070711,-0.0020,0.155689,0.000125,102.0,-0.0200,6.400000,0.0050,0.1550,1.833333,0.1050,0.1570,58.0,65.0,0.022114,4.200000,0.1130,0.0105,0.0050,0.344311,7.000000,0.0050,-0.0220,0.000605,0.004640,36.666667,0.0030,0.0545,0.0210,0.185,0.1600,0.1310,0.000330,0.0050,0.1630,0.600000,0.000800,0.02100,5.238095,-0.9950,0.1070,0.00550,0.215,0.000138,0.514970,0.1650,0.160,0.0055,-5.298317,0.0240,0.0250,0.0890,0.0125,0.008107,0.1155,0.000480,1.00,0.1655,0.000525,0.021,0.0050,1.0,0.0030,0.0030,0.000116,0.1150,0.0050,0.0250,0.185,0.0265,0.0260,0.0050,0.0240,6
3,2,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,2130.0,2.435419,1.502000,0.5390,0.1255,0.500000,-0.0445,0.322763,0.037500,112.0,0.1000,3.966667,0.2500,0.3450,1.827251,0.2250,0.3895,379.0,102.0,0.035397,3.658000,0.6805,0.6255,0.2500,0.349879,4.450122,0.2500,0.0555,0.178362,0.682618,2.311436,0.2055,2.1400,0.9145,0.745,0.2975,1.3895,0.097612,0.1250,0.8005,0.822000,0.148750,0.45725,0.519410,-1.7500,0.2695,0.18775,0.725,0.056325,0.389820,0.8450,0.595,0.3755,-1.386294,1.1200,0.3755,-0.4395,0.4405,0.440061,0.8505,0.122272,2.00,0.9705,0.137175,0.475,0.2500,1.0,0.2055,0.2055,0.343395,0.7250,0.2500,0.2500,0.575,1.2900,1.1645,0.2500,1.1375,10
4,1,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,34.0,2.116373,1.870886,0.4125,0.1720,0.444410,-0.0375,0.835659,0.025675,86.0,0.0675,4.269231,0.1975,0.3575,2.309375,0.2275,0.3950,382.0,588.0,0.038715,3.959494,0.5850,0.5670,0.1975,0.508903,4.887500,0.1975,0.0300,0.157037,0.897435,2.656250,0.1600,1.5560,0.7820,0.685,0.5550,1.2070,0.068000,0.1975,0.7150,0.810127,0.109613,0.78200,0.543478,-0.8025,0.2650,0.36950,0.740,0.048035,0.721447,0.7525,0.555,0

In [8]:
train.Rings = np.log1p(train.Rings)

In [9]:
from autogluon.tabular import TabularDataset, TabularPredictor
from custom_metrics import rmsle_scorer

time_limit = 3600*16

automl = TabularPredictor(label='Rings', problem_type='regression',
                          eval_metric=rmsle_scorer
                          )

automl.fit(train, presets='medium_quality', time_limit=time_limit, num_bag_folds=8, num_bag_sets=0, num_stack_levels=2, dynamic_stacking=False, 
            included_model_types=['XGB', 'CAT', 'GBM', 'XT', 'RF'], ag_args_fit={'num_gpus': 1, 'num_cpus': 8},
          )

No path specified. Models will be saved in: "AutogluonModels\ag-20240420_171305"
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 57600s
AutoGluon will save models to "AutogluonModels\ag-20240420_171305"
=================== System Info ===================
AutoGluon Version:  1.0.1b20240321
Python Version:     3.10.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
Memory Avail:       6.49 GB / 15.42 GB (42.1%)
Disk Space Avail:   529.53 GB / 931.51 GB (56.8%)
Train Data Rows:    94792
Train Data Columns: 86
Label Column:       Rings
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    6639.47 MB
	Train Data (Original)  Memory Usage: 61.83 MB (0.9% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manuall

In [17]:
automl.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L4,-0.044251,mean_squared_log_error,32.427107,1885.998389,0.003976,6.068096,4,True,24
1,WeightedEnsemble_L3,-0.044276,mean_squared_log_error,26.423582,1618.608454,0.002997,3.186564,3,True,16
2,WeightedEnsemble_L2,-0.044288,mean_squared_log_error,12.777632,761.739867,0.003983,2.826576,2,True,8
3,CatBoost_BAG_L2,-0.044342,mean_squared_log_error,13.108655,881.966281,0.053005,52.602862,2,True,12
4,LightGBM_BAG_L2,-0.044347,mean_squared_log_error,13.187657,882.296598,0.132007,52.933179,2,True,10
5,CatBoost_BAG_L3,-0.044375,mean_squared_log_error,26.745604,1783.254404,0.051001,49.344402,3,True,20
6,LightGBMXT_BAG_L3,-0.044381,mean_squared_log_error,26.897600,1789.319443,0.202998,55.409440,3,True,17
7,LightGBMXT_BAG_L2,-0.044385,mean_squared_log_error,13.210649,883.253505,0.155000,53.890086,2,True,9
8,LightGBM_BAG_L3,-0.044407,mean_squared_log_error,26.829609,1787.378722,0.135006,53.468720,3,True,18
9,LightGBMLarge_BAG_L1,-0.044407,mean_squared_log_error,0.315989,121.840561,0.315989,121.840561,1,True,7


In [11]:
predictions = automl.predict(test)

In [12]:
submission = predictions.reset_index().rename(columns={0: 'Rings'})

In [13]:
submission.Rings = np.expm1(submission.Rings)

In [14]:
submission.Rings = np.clip(submission.Rings, 1, 29)

In [15]:
submission.to_csv("submission.csv",index=False)

In [16]:
submission.Rings.mean()

9.58127